Datawhale 十月组队学习：推荐系统

Task01: 推荐系统简介

### 1.推荐系统简介

推荐系统的任务就是联系用户和信息，一方面帮助用户发现对自己有价值的信息，另一方面让信息能够展现在对它感兴趣的用户面前，从而实现信息消费者和信息生产者的双赢。推荐系统通过分析用户的历史行为给用户的兴趣建模，从而主动给用户推荐能够满足他们兴趣和需求的信息。

个性化推荐系统的应用：
* 电子商务
* 电影和视频网站
* 个性化音乐网络平台
* 社交网络
* 个性化阅读
* 个性化广告

好的推荐系统设计，能够让推荐系统本身收集到高质量的用户反馈，不断完善推荐的质量，增加用户和网站的交互，提高网站的收入。

推荐系统实验方法：
* 离线实验
* 用户调查
* 在线实验（AB测试）

### 2.推荐系统常用评价指标

**2.1.用户满意度**

**用户满意度无法离线计算**，只能通过用户调查或者在线实验获得。设计问卷时需要考虑到用户各方面的感受；在在线系统中，用户满意度主要通过一些对用户行为的统计获得，如电商网站可以使用购买率，视频或者内容平台可以设计一些用户反馈界面收集用户满意度，还可以用点击率、用户停留时间和转化率等指标度量用户的满意度。

**2.2.预测准确率**

预测准确度度量一个推荐系统或者推荐算法预测用户行为的能力，这个指标是最重要的推荐系统**离线评测指标**。在计算该指标时需要有一个离线的数据集，该数据集包含用户的历史行为记录。然后将该数据集**通过时间分为训练集和测试集**（如把前6天的数据作为训练集，把第7天的数据作为测试集，为了**避免时间穿越**），最后通过在训练集上建立用户的行为和兴趣模型预测用户在测试集上的行为，并计算预测行为和测试集上实际行为的重合度作为预测准确度。

针对离线推荐系统的不同研究方向有不同的预测准确度指标：

* 评分预测

预测用户对物品的评分行为成为评分预测,评分预测模型通过对用户的历史物品评分记录进行建模,进而得到用户的兴趣模型,然后使用该模型**预测用户未未见过商品的评分**.评分预测的预测准确度一般通过**均方根误差(RMSE)**和**平均绝对误差(MAE)**计算。

对于测试集中的一个用户$u$和物品$i$ ,令$r_{ui}$是用户$u$对物品$i$的实际评分，RMSE和MAE分别定义为：

$$\text{RMSE}=\sqrt{\frac{\sum_{u,i \in T}(r_{ui}-\hat{r}{ui})^2}{|T|}}$$

$$\text{MAE}=\frac{\sum{u,i \in T}\left |r_{ui}-\hat{r}{ui}\right |}{|T|}$$

In [2]:
import math

# 假设用列表 records存放用户评分数据
# records[i] = [u, i, rui, pui]，rui表示用户u对物品i的实际评分，pui是算法预测出来的评分

def RMSE(records):
    return math.sqrt(sum([(rui-pui)**2 for u, i, rui, pui in records]) / float(len(records)))

def MAE(records):
    return sum([abs(rui-pui) for u, i, rui, pui in records]) / float(len(records))

两个指标的优缺点：RMSE加大了对预测不准的用户物品评分的惩罚（平方项的惩罚），因而对系统的评测更加苛刻；如果评分系统是基于整数建立的（即用户给的评分都是整数），那么对预测结果取整会降低MAE的误差。

* TopN 推荐

推荐系统在给用户推荐物品的时候,往往会给用户**一个列表的推荐物品**,这种场景下的推荐成为是TopN推荐,该推荐方式最常用的预测准确率指标一般是**准确率(precision)**和**召回率(recall)**。令$R(u)$是推荐算法根据用户在训练集上的行为给用户作出的推荐列表，$T(u)$是用户在测试集上实际行为过的列表，那么推荐的准确率和召回率分别定为为：

$$\text{Precision}=\frac{\sum_{u \in U}\left | R(u) \cap T(u) \right |}{\sum_{u \in U} |R(u)|}$$


$$\text{Recall}=\frac{\sum{u \in U}\left |R(u) \cap T(u)\right |}{\sum_{u \in U} |T(u)|} $$

有时为了全面评测TopN推荐的准确率和召回率，一般会选取不同的推荐列表长度N，计算出一组Precision和Recall，然后画出PR曲线。

> 关于准确率、召回率，PR曲线，ROC曲线、AUC的知识可以参考：[【机器学习】一文读懂分类算法常用评价指标](https://www.cnblogs.com/guoyaohua/p/classification-metrics.html)

In [4]:
def PrecisionRecall(test, N):
    hit = 0
    n_recall = 0
    n_precision = 0
    for user, items in test.items():
        rank = Recommend(user, N)
        hit += len(rank & items)      #预测为正，实际为正
        n_recall += len(items)        #实际为正
        n_precision += N              #预测为正
    return [hit / (1.0 * n_recall), hit / (1.0 * n_precision)]

* 覆盖率

覆盖率是用来描述一个推荐系统对物品长尾的发掘能力,一个简单的定义可以是:推荐系统所有推荐出来的物品集合数占总物品集合数的比例。设用户集合为$U$，推荐系统给每个用户推荐一个长度为N的物品列表$R(u)$，那么覆盖率可由下式计算：

$$\text{Coverage}=\frac{\left|U_{u \in U}R(u)\right|}{|I|}$$

为了更细致地描述推荐系统发掘长尾物品的能力，需要统计推荐列表中不同物品出现次数的分布，如果所有物品都出现在推荐列表中，且出现次数不多，则推荐系统发掘长尾的能力就很好。在信息论和经济学中有两个著名的指标可以用来定义覆盖率，第一个是**信息熵**，其中$p(i)$是物品$i$的流行度除以所有物品流行度之和：

> 物品的流行度是指对物品产生过行为的用户总数。

$$ H=-\sum\limits_{i=1}^n p(i)\log p(i) $$

第二个是**基尼系数**（Gini Index），其中$i_j$是按照物品流行度$p$从小到大排序的物品列表中第$j$个物品：

$$G= \frac{1}{n-1} \sum_{j=1}^n (2j - n -1) p(i_j)$$

In [5]:
def GiniIndex(p):
    j = 1
    n = len(p)
    G = 0
    for item, weight in sorted(p.items(), key=itemgetter(1)):
        G += (2 * j - n - 1) * weight
    return G / float(n - 1)

> **马太效应**即强者更强，弱者更弱的效应。如果一个系统会增大热门物品和非热门物品的流行度差距，让热门的物品更加热门，不热门的物品更加不热门，那么这个系统就有马太效应。

评测推荐系统是否具有马太效应的简单方法是使用基尼系数，如果$G1$是从初始用户行为中计算出来的物品流行度的基尼系数，$G2$是从推荐列表中计算出来的物品流行度的基尼系数，那么如果$G2>G1$，就说明推荐算法具有马太效应。

* 多样性

用户的兴趣是广泛的，为了满足用户广泛的兴趣，推荐列表需要能够覆盖用户不同的兴趣领域，即推荐结果需要具有多样性。尽管用户的兴趣在较长时间跨度中是不一样的，但具体到用户访问推荐系统的某一刻，其兴趣往往是单一的，如果推荐列表比较多样，覆盖了用户绝大多数的兴趣点，就会增加用户找到感兴趣物品的概率。

多样性描述了推荐列表中物品两两之间的不相似性，假设$s(i, j \in [0,1$定义了物品$i$和$j$之间的相似度，那么用户$u$的推荐列表$R(u)$的多样性定义如下：

$$\text{Diversity}(R(u))=1-\frac{\sum_{i,j \in R(u), i \neq j} s(i, j)}{\displaystyle \frac{1}{2} \left|R(u)\right|(\left|R(u)\right|-1)}$$ 

推荐系统的整体多样性可以定义为所有用户推荐列表多样性的平均值： 

$$\text{Diversity}=\frac{1}{\left|U\right|}\sum_{u \in U} \text{Diversity}(R(u))$$

* 新颖性

新颖的推荐是指给用户推荐那些他们以前没有听说过的物品。实现新颖性的简单办法是，把那些用户之前在网站中对其有过行为的物品从推荐列表中过滤掉。评测新颖度的最简单方法是利用推荐系统的平均流行度，因为越不热门的物品可能让用户觉得新颖。目前研究的难点是如何在不牺牲精度的情况下提高多样性和新颖性。

* 惊喜度

要分清惊喜度（serendipity）与新颖性的区别，如果推荐结果和用户的历史兴趣不相似，但却让用户觉得满意，那可以说推荐结果的惊喜度很高；而推荐的新颖性仅仅取决于用户是否听说过这个推荐结果。

* 信任度

让用户对推荐系统产生信任是很重要的，提高推荐系统信任度的方法有两种：增加推荐的透明度（transparency），提供推荐解释；考虑用户的社交网络信息，利用用户的好友信息给用户作推荐，并且用好友进行推荐解释。

* 实时性

很多物品（新闻，微博等）具有很强的时效性，推荐系统的实时性包括两个方面：推荐系统需要实时地更新推荐列表来满足用户新的行为变化；推荐系统需要能够将新加入系统的物品推荐给用户，即处理物品冷启动的能力。

* 健壮性

健壮性（robust，即鲁棒性）衡量了一个推荐系统抗击作弊的能力，健壮性的评测主要利用模拟攻击，用作弊算法给数据集注入噪声。提高健壮性的方法有：尽量使用代价比较高的用户行为，如购买行为>浏览行为；使用数据前进行攻击检测。

* 商业目标

推荐系统的“终极”优化目标包括两个维度：一个维度是用户体验的优化；另一个维度是满足公司的商业利益，这两个维度应该是和谐统一的。不同的网站有不同的商业目标，如youtube主要的优化目标是观看时长，电商网站的优化目标可以是购买转化率。

* AUC

这个知识点在之前[贷款违约预测](https://nekomoon404.github.io/2020/09/15/%E8%B4%B7%E6%AC%BE%E8%BF%9D%E7%BA%A6%E9%A2%84%E6%B5%8B%EF%BC%881%EF%BC%89%E8%B5%9B%E9%A2%98%E7%90%86%E8%A7%A3/)学习中已经有过较详细的了解了，这里就不赘述了。（其实是懒(´⊙ω⊙`)）

> 可参考：王喆-《深度学习推荐系统》-7.2 直接评估推荐序列的离线指标 


### 3.推荐系统召回层

#### 3.1.召回层的功能特点

推荐系统的“模型部分”是推荐系统的主体，模型的结果一般由“召回层”，“排序层”，“补充策略层”组成。

* 召回层：一般利用高效的召回规则、算法或简单的模型，快速从海量的候选集中召回用户可能感兴趣的物品

* 排序层：利用排序模型对初筛的候选集进行精排序

* 补充策略与算法层：也被称为再排序层，在将推荐列表返回用户之前，为兼顾结果的多样性，流行度，新鲜度等指标，结合一些补充的策略和算法对推荐列表进行一定的调整，最终形成用户可见的推荐列表。

推荐系统的模型部分将推荐过程分为召回层和排序层的主要原因是基于过程上的考虑，总结下召回层和排序层的特点：

* 召回层：待计算的候选集合大、计算速度快、模型简单、特征较少，尽量让用户感兴趣的物品在这个阶段能够被快速召回，即保证相关物品的召回率

* 排序层：首要目标是得到精准的排序结果。需要处理的物品数量少，可以利用较多的特征，使用比较复杂的模型

#### 3.2.多路召回策略
为了权衡计算速度和召回率，目前工业界主流的召回方法是采用多个简单策略叠加的“**多路召回策略**”，它是指采用不同的策略、特征或简单模型，分别召回一部分候选集，然后把候选集混合在一起供后续排序模型使用的策略。其中各简单策略保证候选集的快速召回，从不同角度设计的策略保证召回率接近理想的状态，不至于损害排序效果。

具体使用哪些召回策略是与业务强相关的，对于每一路召回都会从商品集合中拉回$K$个商品，这里的$K$是一个超参数，对于$K$的选择一般需要通过离线评估加线上的A/B测试来确定合理的$K$值。

多路召回也存在缺点，从策略选择到候选集大小参数的调整都需要人工参与，策略之间的信息也是割裂的，无法综合考虑不同策略对一个物品的影响。

#### 3.3.基于embedding的召回方法

基于Embedding的召回方法，是一个综合性强且计算速度也能满足需求的召回方法。多路召回中使用“兴趣标签”“热门度”“流行趋势”“物品属性”等信息都可以作为Embedding召回方法中的附加信息融合进最终的Embedding向量中， 这就相当于考虑了多路召回的多种策略。

Embedding的另一个优势在于评分的连续性，可以把Embedding间的相似度作为唯一的判断标准，因此可以随意限定召回的候选集大小。

> 自己对于召回和Embedding这两块的内容都还不熟悉，后续还需要再深入学习。


### 4.Task01 小结

* 了解了推荐系统的基础知识和主要技术框架
* 学习推荐系统的各种评测指标
* 对推荐系统召回层的特点和策略有所了解

************************
参考资料：

1. 项亮-《推荐系统实践》-第一章 好的推荐系统

2. 王喆-《深度学习推荐系统》-第1,4,5,7章



扩展阅读（收藏夹吃灰）：

 1. [关于推荐系统的入门知识一些整理(一)](https://zhuanlan.zhihu.com/p/140539453) 

 2. [《推荐系统实践》读书笔记](https://relph1119.github.io/recommendation-system-practice-notes/#/README)

 3. [深入理解推荐系统：召回](https://zhuanlan.zhihu.com/p/115690499)

 4. [Embedding在深度推荐系统中的3大应用方向](https://zhuanlan.zhihu.com/p/67218758)

 5. [推荐系统Embedding向量召回在即刻的工程实践](https://zhuanlan.zhihu.com/p/160120292)